In [1]:
# 코드 7-19 라이브러리 호출
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [2]:
# 코드 7-20 데이터셋 내려받기 및 전처리
start = time.time()
TEXT = torchtext.legacy.data.Field(sequential=True, batch_first=True, lower=True)
LABEL = torchtext.legacy.data.Field(sequential=False, batch_first=True)

from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)  # 역시 IMDB를 사용합니다.
train_data, valid_data = train_data.split(split_ratio=0.8)

TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

BATCH_SIZE = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# 코드 7-21 데이터셋 분리
train_iterator, valid_iterator, test_iterator = (
    torchtext.legacy.data.BucketIterator.splits(
        (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device
    )
)

In [4]:
# 코드 7-22 변수 값 지정
vocab_size = len(TEXT.vocab)
n_classes = 2  # pos(긍정), neg(부정)

In [5]:
# 코드 7-23 RNN 계층 네트워크
class BasicRNN(nn.Module):
    def __init__(
        self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2
    ):
        super(BasicRNN, self).__init__()
        self.n_layers = n_layers  # RNN 계층에 대한 개수
        self.embed = nn.Embedding(n_vocab, embed_dim)  # 워드 임베딩 적용
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)  # 드롭아웃 적용
        self.rnn = nn.RNN(
            embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True
        )
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)  # 문자를 숫자/벡터로 변환
        h_0 = self._init_state(
            batch_size=x.size(0)
        )  # 최초 은닉 상태의 값을 0으로 초기화
        x, _ = self.rnn(
            x, h_0
        )  # RNN 계층을 의미하며, 파라미터로 입력과 이전 은닉 상태의 값을 받습니다.
        h_t = x[
            :, -1, :
        ]  # 모든 네트워크를 거쳐서 가장 마지막에 나온 단어의 임베딩 값(마지막 은닉 상태의 값)
        self.dropout(h_t)
        logit = torch.sigmoid(self.out(h_t))
        return logit

    def _init_state(self, batch_size=1):
        weight = next(
            self.parameters()
        ).data  # 모델의 파라미터 값을 가져와서 weight 변수에 저장
        return weight.new(
            self.n_layers, batch_size, self.hidden_dim
        ).zero_()  # 크기가 (계층의 개수, 배치 크기, 은닉층의 뉴런/유닛 개수)인 은닉 상태(텐서)를 생성하여 0으로 초기화한 후 반환

In [6]:
# 코드 7-24 손실 함수와 옵티마이저 설정
model = BasicRNN(
    n_layers=1,
    hidden_dim=256,
    n_vocab=vocab_size,
    embed_dim=128,
    n_classes=n_classes,
    dropout_p=0.5,
)
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [20]:
# 코드 7-25 모델 학습 함수
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

        if b % 50 == 0:  # 훈련 데이터셋의 개수를 50으로 나누어서 나머지가 0이면 출력
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    e,
                    b * len(x),
                    len(train_iterator.dataset),
                    100.0 * b / len(train_iterator),
                    loss.item(),
                )
            )

In [21]:
# 코드 7-26 모델 평가 함수
def evaluate(model, val_iter):
    model.eval()
    corrects, total, total_loss = 0, 0, 0

    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction="sum")
        total += y.size(0)
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [22]:
# 코드 7-27 모델 학습 및 평가
BATCH_SIZE = 100
LR = 0.001
EPOCHS = 5
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iterator)
    val_loss, val_accuracy = evaluate(model, valid_iterator)
    print(
        "[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f"
        % (e, val_loss, val_accuracy)
    )

Train Epoch: 1 [0/20000 (0%)]	Loss: 0.693566
Train Epoch: 1 [5000/20000 (25%)]	Loss: 0.695677
Train Epoch: 1 [10000/20000 (50%)]	Loss: 0.692407
Train Epoch: 1 [15000/20000 (75%)]	Loss: 0.689900
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy:  0.51
Train Epoch: 2 [0/20000 (0%)]	Loss: 0.693259
Train Epoch: 2 [5000/20000 (25%)]	Loss: 0.692251
Train Epoch: 2 [10000/20000 (50%)]	Loss: 0.690142
Train Epoch: 2 [15000/20000 (75%)]	Loss: 0.701210
[EPOCH: 2], Validation Loss:  0.69 | Validation Accuracy:  0.50
Train Epoch: 3 [0/20000 (0%)]	Loss: 0.694055
Train Epoch: 3 [5000/20000 (25%)]	Loss: 0.699353
Train Epoch: 3 [10000/20000 (50%)]	Loss: 0.692231
Train Epoch: 3 [15000/20000 (75%)]	Loss: 0.693730
[EPOCH: 3], Validation Loss:  0.69 | Validation Accuracy:  0.50
Train Epoch: 4 [0/20000 (0%)]	Loss: 0.693572
Train Epoch: 4 [5000/20000 (25%)]	Loss: 0.697761
Train Epoch: 4 [10000/20000 (50%)]	Loss: 0.692645
Train Epoch: 4 [15000/20000 (75%)]	Loss: 0.694609
[EPOCH: 4], Validation Loss:  0.

In [23]:
# 코드 7-28 테스트 데이터셋을 이용한 모델 예측
test_loss, test_acc = evaluate(model, test_iterator)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

Test Loss:  0.69 | Test Accuracy:  0.54
